In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import requests
from PIL import Image
from io import BytesIO
import os
from webdriver_manager.chrome import ChromeDriverManager

chrome_option = Options()
chrome_option.add_experimental_option('detach', True)  
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_option)

def save_image(url, folder_path, index):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img.save(os.path.join(folder_path, f"image_{index}.png"))
        print(f"Saved image {index} to {os.path.join(folder_path, f'image_{index}.png')}")
    except Exception as e:
        print(f"Error saving image {index}: {e}")

def download_photo_reviews_from_product(save_dir):
    try:
        review_button = driver.find_element(By.CLASS_NAME, 'review_btn_more__OL2sc')
        review_button.click()
        time.sleep(3)  

        photo_reviews = driver.execute_script("""
            return Array.from(document.querySelectorAll(".reviewItems_img_box__Od5DB img")).map(x => x.src);
        """)
        print(photo_reviews)

        os.makedirs(save_dir, exist_ok=True)
        
        for j, url in enumerate(photo_reviews):
             save_image(url, save_dir, f"{j+1}")

    except Exception as e:
        print(f"Error processing product: {e}")

def process_products_from_search_page(search_url):
    driver.get(search_url)
    time.sleep(3)  

    page_number = 1
    while True:
        print(f"Processing page {page_number}...")
        product_links = driver.find_elements(By.CLASS_NAME, 'product_compare__Kfwlg.linkAnchor')
        if not product_links:
            print("No more products found.")
            break
        
        for i, link in enumerate(product_links):
            try:
                link.click()
                time.sleep(3)
                driver.switch_to.window(driver.window_handles[-1])
                
                save_dir = f'NaverShoping_Data_/Page_{page_number}/Product_{i+1}'
                download_photo_reviews_from_product(save_dir)
                
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(3)

            except Exception as e:
                print(f"Error processing product {i}: {e}")

        try:
            next_button = driver.find_element(By.CLASS_NAME, 'pagination_next__pZuC6')
            if 'disabled' in next_button.get_attribute('class'):
                print("No more pages.")
                break
            next_button.click()
            time.sleep(3)  
            page_number += 1

        except Exception as e:
            print(f"Error processing product {i}: {e}")

        try:
            next_button = driver.find_element(By.CLASS_NAME, 'pagination_next__pZuC6')
            if 'disabled' in next_button.get_attribute('class'):
                print("No more pages.")
                break
            next_button.click()
            time.sleep(3)  
            page_number += 1

        except Exception as e:
            print(f"Error navigating to next page: {e}")
            break

search_url = 'https://search.shopping.naver.com/search/all?query=%EC%9D%98%EB%A5%98'
while True:
    try:
        process_products_from_search_page(search_url)
    except Exception as e:
        print(f"Error in main loop: {e}")
    finally:
        driver.quit()
        time.sleep(60)
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_option)

Processing page 1...
['https://phinf.pstatic.net/checkout/20230521_293/1684623762148WPcme_JPEG/20230517_160822.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20230521_7/1684623763328Jt08t_JPEG/20230517_160552.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20230521_85/1684623763963jP6yj_JPEG/20230517_160535.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20230521_164/1684623764881vcjpm_JPEG/20230517_160635.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20240606_67/1717626710670fjCrH_JPEG/image.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20240606_156/17176267249021zaBT_JPEG/image.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20240606_229/1717626752215Ct89r_JPEG/image.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20240607_152/17177576765902CN2g_JPEG/20240607_195001.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20240529_9/1716948958015Gw9Yb_JPEG/KakaoTalk_20240528_171116462.jpg?type=f287', 'https://phinf.pstatic.net/checkout/20240517_275/1715939